In [ ]:
!pip install --upgrade transformers

In [ ]:
import transformers

device='cuda'
max_len=64
train_batch_size=8
valid_batch_size=4
epochs=10
bert_path='bert-base-uncased'
model_path='models/bert_sentiment'
training_file='/content/IMDB Dataset.csv'
tokenizer=transformers.BertTokenizer.from_pretrained(bert_path,do_lower_case=True)


In [ ]:
import torch

class BERTDataset:
  def __init__(self,review,target):
    self.review=review
    self.target=target
    self.tokenizer=tokenizer
    self.max_len=max_len

  def __len__(self):
    return len(self.review)

  def __getitem__(self,item):
    review=str(self.review[item])
    review=' '.join(review.split())

    inputs=self.tokenizer.encode_plus(review,
                                      None,
                                      add_special_tokens=True,
                                      max_length=self.max_len,
                                      padding='max_length',
                                      truncation=True)
    ids=inputs.get('input_ids', inputs.get('inputs_ids'))
    mask=inputs['attention_mask']
    token_type_ids=inputs['token_type_ids']

    return {
        'ids':torch.tensor(ids,dtype=torch.long),
        'mask':torch.tensor(mask,dtype=torch.long),
        'token_type_ids':torch.tensor(token_type_ids,dtype=torch.long),
        'targets':torch.tensor(self.target[item],dtype=torch.float)
    }


In [ ]:
import transformers
import torch.nn as nn

class BERTBaseUncased(nn.Module):
  def __init__(self):
    super(BERTBaseUncased,self).__init__()
    self.bert=transformers.BertModel.from_pretrained(bert_path)
    self.bert_drop=nn.Dropout(0.3)
    self.out=nn.Linear(768,1)
  def forward(self,ids,mask,token_type_ids):
    o2=self.bert(ids,attention_mask=mask,token_type_ids=token_type_ids).last_hidden_state[:,0,:]
    bo=self.bert_drop(o2)
    output=self.out(bo)
    return output

In [ ]:
import torch
import torch.nn
from tqdm import tqdm

def loss_fn(outputs,targets):
  return torch.nn.BCEWithLogitsLoss()(outputs,targets.view(-1,1))

def train_fn(data_loader,model,optimizer,device,scheduler):
  model.train()
  # Wrap the data_loader with tqdm to display the progress bar
  for bi,d in tqdm(enumerate(data_loader,len(data_loader))):
    ids=d['ids']
    token_type_ids=d['token_type_ids']
    mask=d['mask']
    targets=d['targets']

    ids=ids.to(device,dtype=torch.long)
    token_type_ids=token_type_ids.to(device,dtype=torch.long)
    mask=mask.to(device,dtype=torch.long)
    targets=targets.to(device,dtype=torch.float)

    optimizer.zero_grad()
    outputs=model(ids=ids,mask=mask,token_type_ids=token_type_ids)

    loss=loss_fn(outputs,targets)
    loss.backward()
    optimizer.step()
    scheduler.step

def eval_fn(data_loader,model,device):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  with torch.no_grad():
    # Wrap the data_loader with tqdm to display the progress bar
    for bi,d in tqdm(enumerate(data_loader,len(data_loader))):
      ids=d['ids']
      token_type_ids=d['token_type_ids']
      mask=d['mask']
      targets=d['targets']

      ids=ids.to(device,dtype=torch.long)
      token_type_ids=token_type_ids.to(device,dtype=torch.long)
      mask=mask.to(device,dtype=torch.long)
      targets=targets.to(device,dtype=torch.float)

      outputs=model(ids=ids,mask=mask,token_type_ids=token_type_ids)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
  return fin_outputs,fin_targets

In [ ]:

import torch
import pandas as pd
import torch.nn as nn
import numpy as np
from sklearn import model_selection
from sklearn import metrics
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup


def run():
    dfx = pd.read_csv(training_file).fillna("none")
    dfx.sentiment = dfx.sentiment.apply(lambda x: 1 if x == "positive" else 0)

    df_train, df_valid = model_selection.train_test_split(
        dfx, test_size=0.1, random_state=42, stratify=dfx.sentiment.values
    )

    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)

    train_dataset = BERTDataset(
        review=df_train.review.values, target=df_train.sentiment.values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=train_batch_size, num_workers=4
    )

    valid_dataset = BERTDataset(
        review=df_valid.review.values, target=df_valid.sentiment.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=valid_batch_size, num_workers=1
    )

    #device = torch.device(device)
    model = BERTBaseUncased()
    model.to(device)

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]

    num_train_steps = int(len(df_train) / train_batch_size * epochs)
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
    )

    best_accuracy = 0
    for epoch in range(epochs):
        train_fn(train_data_loader, model, optimizer, device, scheduler)
        outputs, targets = eval_fn(valid_data_loader, model, device)
        outputs = np.array(outputs) >= 0.5
        accuracy = metrics.accuracy_score(targets, outputs)
        print(f"Accuracy Score = {accuracy}")


if __name__ == "__main__":
    run()

In [ ]:
df=pd.read_csv("/content/IMDB Dataset.csv")

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
new_str=df['review'][0]

In [ ]:
new_str

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [ ]:
def __getitem__(review):
    review=' '.join(review.split())

    inputs=tokenizer.encode_plus(review,
                                      None,
                                      add_special_tokens=True,
                                      max_length=max_len,
                                      padding='max_length',
                                      truncation=True)
    ids=inputs.get('input_ids', inputs.get('inputs_ids'))
    mask=inputs['attention_mask']
    token_type_ids=inputs['token_type_ids']

    return {
        'ids':torch.tensor(ids,dtype=torch.long),
        'mask':torch.tensor(mask,dtype=torch.long),
        'token_type_ids':torch.tensor(token_type_ids,dtype=torch.long)
        }

In [ ]:
token_str=__getitem__(new_str)
token_str

{'ids': tensor([  101,  2028,  1997,  1996,  2060, 15814,  2038,  3855,  2008,  2044,
          3666,  2074,  1015, 11472,  2792,  2017,  1005,  2222,  2022, 13322,
          1012,  2027,  2024,  2157,  1010,  2004,  2023,  2003,  3599,  2054,
          3047,  2007,  2033,  1012,  1026,  7987,  1013,  1028,  1026,  7987,
          1013,  1028,  1996,  2034,  2518,  2008,  4930,  2033,  2055, 11472,
          2001,  2049, 24083,  1998,  4895, 10258,  2378,  8450,  5019,  1997,
          4808,  1010,  2029,   102]),
 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [ ]:
outputs_1=model(ids=token_str['ids'].unsqueeze(0),mask=token_str['mask'],token_type_ids=token_str['token_type_ids'])

In [ ]:
outputs_1